## Optimization
### Newton's method
In **Newton's method**, the inverse of **Hessian** matrix multiplied by the negative of **gradient** of the given objective function $F(\boldsymbol{x})$ is used to update the current guess $\boldsymbol{x_k}$:
<br>$\boldsymbol{x}_{k+1}=\boldsymbol{x}_k-\eta_k H_F(\boldsymbol{x}_k)^{-1}\nabla F(\boldsymbol{x}_k)$
<br> where $\eta_k>0$ is the **learning rate** (also called *step size*) at time step $k$. 
<br>**Hint 1:** In the original formulation of Newton's method, $\eta_k=1$ for all $k$.
<br> **Hint 2:** If the initial guess $\boldsymbol{x}_0$ is close enough to the minimum point and the Hessian is **nonsingular**, then the sequences $x_k$ converges to the minimizer. During these iterations, we must have a **positive definite** Hessian matrix to get to the minimizer.
<br> **Reminder 1:** A square matrix is nonsingular if it has *inverse*. In fact, a matrix is nonsingular if and only if its *determinant* is nonzero.
<br> **Reminder 2:** A *symmetric square* matrix, say $A$, is *positive definite* if and only if: 
- $\boldsymbol{x}^TA\;\boldsymbol{x}>0 \;\; \forall \boldsymbol{x}\in R^q-\{0\}$
<br><br>**Contents**
- The Python code of the Newton's method for optimization
- Example with Booth function (convex function)
- Example with Beale function (nonconvex function)
    - With the initial guess near enough to the minimizer
    - With the initial guess not close enough to the minimizer
    
<hr>
The Python code at: https://github.com/ostad-ai/Optimization
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/Optimization

In [1]:
# importing required modules
import numpy as np

In [2]:
# The function that we are going to optimize (minimize)
# https://en.wikipedia.org/wiki/Test_functions_for_optimization
# Booth is convex and has a minimum at f(1,3)=0
def Booth(xs):
    x,y=xs
    f=(x+2*y-7)**2+(2*x+y-5)**2
    return f

# Beale is nonconvex and its global minimum is at f(3,0.5)=0
def Beale(xs):
    x,y=xs
    f=(1.5-x+x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)**2
    return f

# first order partial derivatives
def partial(f,x0,i,h=.001):
    x2=x0.copy(); x1=x0.copy()
    x2[i]+=h; x1[i]-=h
    diff=f(x2)-f(x1)
    return diff/(2.*h)

# second order partial derivatives
def partial2(f,x0,i,j,h=.001):
    x2=x0.copy(); x1=x0.copy()
    x2[i]+=h; x1[i]-=h
    f2=partial(f,x2,j,h)
    f1=partial(f,x1,j,h)
    return (f2-f1)/(2.*h)

# returns gradient in a column vector
def gradient(f,x,h=.001):
    q=x.shape[0]
    gradf=np.zeros(q)
    for i in range(q):
        gradf[i]=partial(f,x,i,h)
    return gradf.reshape(-1,1)

# returns the Hessian matrix of function f
def hessian(f,x,h=.001,landa=.0):
    q=x.shape[0]
    hMat=np.zeros((q,q)) #Hessian matrix
    for i in range(q):
        for j in range(q):
            hMat[i,j]=partial2(f,x,i,j,h)
            if i==j: 
                # to make it positive definite
                hMat[i,j]+=landa 
    return hMat

In [3]:
# the newton method for optimization
def newtonOpt(x0,f,iter=100,etta=.05):
    x=x0.copy().astype('float')
    for i in range(iter):
        hg=np.linalg.inv(hessian(f,x))@gradient(f,x)
        x-=etta*hg
    return x.flatten(),f(x)

Since the Booth function is **quadaric**, the Newton's method can get to the *minimizer* in just one iteration with $\eta=1$. Let's try this in the following code:

In [4]:
# Example: finding minimum of Booth function in one iteration
x0=np.array([0.,0.]).reshape(-1,1) # initial point or guess 
x_final,fitness=newtonOpt(x0,Booth,iter=1,etta=1)
print('--- Newton\'s method for Booth function ---')
print(f'Initial guess={x0.flatten()}')
print(f'Final solution={x_final} \nwith fitness={fitness}')

--- Newton's method for Booth function ---
Initial guess=[0. 0.]
Final solution=[1. 3.] 
with fitness=[3.88235686e-18]


The Beale function is not convex, so if the initial guess is far from the optimium $x=[3,0.5]$, it gets stuck in a local minimum. In the following example, we choose a close enough initial guess for the experiment.

In [5]:
# Example: finding minimum of Beale function
# we choose a close enough initial guess to the true optimum [3,.5]
x0=np.array([2.,0.]).reshape(-1,1) # initial point or guess 
x_final,fitness=newtonOpt(x0,Beale)
print('--- Newton\'s method for Beale function ---')
print(f'Initial guess={x0.flatten()}')
print(f'Final solution={x_final} \nwith fitness={fitness}')

--- Newton's method for Beale function ---
Initial guess=[2. 0.]
Final solution=[2.97581249 0.49372117] 
with fitness=[9.72895881e-05]


In the example below, for the **Beale** function, we start from an initial guess not close enough to the minimizer. 
So, it gets stuck in a **stationary point**, which is a **saddle point** here.

In [6]:
# Example: finding minimum of Beale function
# This time we choose a little far initial guess to the true optimum [3,.5]
x0=np.array([0.,0.]).reshape(-1,1) # initial point or guess 
x_final,fitness=newtonOpt(x0,Beale)
print('--- Newton\'s method for Beale function ---')
print(f'Initial guess={x0.flatten()}')
print(f'Final solution={x_final} \nwith fitness={fitness}')
print('This time, it has converged to the saddle point (stationary point): [0,1]')

--- Newton's method for Beale function ---
Initial guess=[0. 0.]
Final solution=[0.         0.99744639] 
with fitness=[14.203125]
This time, it has converged to the saddle point (stationary point): [0,1]
